<a href="https://colab.research.google.com/github/DylanCTY/TextAnalytics_LearningSpace/blob/main/IB9CW0_5504008_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install Ollama

In [1]:
# Install Ollama v0.1.40
!curl https://ollama.ai/install.sh | sed 's#https://ollama.ai/download#https://github.com/jmorganca/ollama/releases/download/v0.1.40#' | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  25878      0 --:--:-- --:--:-- --:--:-- 25926
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Get Ollama running in the background of the Colab

In [2]:
# Set up Ollama as a global variable and run it in the background
OLLAMA_MODEL='phi:latest'
import os
os.environ['OLLAMA_MODEL'] = OLLAMA_MODEL
!echo $OLLAMA_MODEL # Verify the global variable is set

import subprocess
import time

# Start ollama on the server ("serve")
command = "nohup ollama serve&" # "nohup" and "&" means run in the background

# Use subprocess.Popen to run the command
process = subprocess.Popen(command,
                            shell=True,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE)

print("Process ID:", process.pid) # Print the process ID
time.sleep(5)  # Makes Python wait for 5 seconds

!ollama -v # print the Ollama version number as a check

phi:latest
Process ID: 1031
ollama version is 0.1.41


Query the model to generate text about football games in the UK from 2004 to 2005

In [3]:
# Query the model via the command line
!ollama run $OLLAMA_MODEL "Briefly introduce the football games in the UK from 2004 to 2005."

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 04778965089b...   0% ▕▏    0 B/1.6 GB                  pulling manifest 
pulling 047789650

Start to build our RAG. Firstly, this involves installing a bunch of packages:

In [4]:
# Install necessary packages
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-ollama
!pip install llama-index-vector-stores-chroma
!pip install llama-index ipywidgets
!pip install llama-index-llms-huggingface
!pip install chromadb

# Import required modules from the llama_index library
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.readers.file import FlatReader
from llama_index.core import Document
from llama_index.core.schema import TextNode

# Import the Ollama class
from llama_index.llms.ollama import Ollama

# Import ChromaVectorStore and chromadb module
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

# Additional imports
from zipfile import ZipFile
import os
from pathlib import Path
from transformers import AutoModel, AutoTokenizer
from numpy.linalg import norm
import torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.2/853.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  

Read in the text and implement the chunking

In [5]:
# Unzip the dataset
zip_file_path = '/content/bbcsport_football.zip'  # Path to the uploaded zip file
unzip_dir = '/content/bbcsport_football'
with ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Combine the content of all text files into a single string, excluding hidden and system files
combined_text = ""
for root, dirs, files in os.walk(unzip_dir):
    for filename in sorted(files):
        if filename.endswith(".txt") and not filename.startswith('._') and not filename.startswith('__MACOSX'):
            with open(os.path.join(root, filename), 'r', encoding='utf-8', errors='ignore') as file:
                combined_text += file.read() + "\n"

# Replace "\n" (paragraph break) and "\t" (tab character)
combined_text = combined_text.replace("\n", "")
combined_text = combined_text.replace("\t", "")

# Write the combined text to a temporary file
temp_file_path = '/content/combined_bbcsport.txt'
with open(temp_file_path, 'w', encoding='utf-8') as temp_file:
    temp_file.write(combined_text)

# Read the combined text to verify its content
with open(temp_file_path, 'r', encoding='utf-8') as temp_file:
    content = temp_file.read()
    print(f"Combined text length: {len(content)} characters")
    print(f"First 1000 characters of the combined text:\n{content[:1000]}")

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

1-1. Semantic Chunking (model: BAAI/bge-small-en-v1.5)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Convert nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes)]

# Check if docs is defined and not empty
print(f"Total docs created: {len(docs)}")

# Print the first few docs to verify their content
for i in range(min(5, len(docs))):
    print(f"Doc {i+1} text:\n{docs[i].text}\n")

Combined text length: 516804 characters
First 1000 characters of the combined text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete co

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Total nodes created: 402
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Good

1-2. Semantic Chunking (model: jinaai/jina-embeddings-v2-small-en)

In [ ]:
# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Specify the embedding model into LlamaIndex's settings
Settings.embed_model = embed_model

# Initialize the SemanticSplitterNodeParser with desired settings
parser = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=93, embed_model=embed_model
)

# Get nodes using the semantic chunking model
bbcsport_nodes = parser.get_nodes_from_documents(bbcsport_docs)

# Convert nodes to the expected document type (TextNode)
docs = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes)]

# Check if docs is defined and not empty
print(f"Total docs created: {len(docs)}")

# Print the first few docs to verify their content
for i in range(min(5, len(docs))):
    print(f"Doc {i+1} text:\n{docs[i].text}\n")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-small-en and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.2.intermediate.dense.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.2.output.LayerNorm.bias', 'encoder.layer.2.output.LayerNorm.weight', 'encoder.layer.2.output.dense.bias', 'encoder.layer.2.output.dense.weight', 'encoder.layer.3.intermediate.dense.bias', 'encoder.layer.3.intermediate.den

Total docs created: 282
Doc 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Goodis

2. Sentence Splitter

In [6]:
# Define the TextNode class if not already defined
class TextNode:
    def __init__(self, text, doc_id):
        self.text = text
        self.doc_id = doc_id

# Load the data from the combined text file
bbcsport_docs = FlatReader().load_data(Path(temp_file_path))

# We will limit to chunk size 100
parser = SentenceSplitter(chunk_size=200, chunk_overlap=50)

# Get nodes using the sentence splitting model
bbcsport_nodes_raw = parser.get_nodes_from_documents(bbcsport_docs)

# Convert raw nodes to the expected document type (TextNode)
bbcsport_nodes = [TextNode(text=node.text, doc_id=str(i)) for i, node in enumerate(bbcsport_nodes_raw)]

# Check if bbcsport_nodes is defined and not empty
print(f"Total nodes created: {len(bbcsport_nodes)}")

# Print the first few nodes to verify their content
for i in range(min(5, len(bbcsport_nodes))):
    print(f"Node {i+1} text:\n{bbcsport_nodes[i].text}\n")


Total nodes created: 862
Node 1 text:
Man Utd stroll to Cup winWayne Rooney made a winning return to Everton as Manchester United cruised into the FA Cup quarter-finals.Rooney received a hostile reception, but goals in each half from Quinton Fortune and Cristiano Ronaldo silenced the jeers at Goodison Park. Fortune headed home after 23 minutes before Ronaldo scored when Nigel Martyn parried Paul Scholes' free-kick. Marcus Bent missed Everton's best chance when Roy Carroll, who was later struck by a missile, saved at his feet.Rooney's return was always going to be a potential flashpoint, and he was involved in an angry exchange with a spectator even before kick-off. And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old.

Node 2 text:
And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a hea

Check the output

In [7]:
# Access the second node's text
print(bbcsport_nodes[1].text)

And Rooney's every touch was met with a deafening chorus of jeers from the crowd that once idolised the 19-year-old. Everton started brightly and Fortune needed to be alert to scramble away a header from Bent near the goal-line. But that was the cue for United to take complete control with a supreme passing display on a Goodison Park pitch that was cutting up. Fortune gave United the lead after 23 minutes, rising to meet Ronaldo's cross from eight yards after the Portuguese youngster had been allowed too much time and space by the hapless Gary Naysmith. United dominated without creating too many clear-cut chances, and they almost paid the price for not making the most of their domination two minutes before half-time. Mikel Arteta played a superb ball into the area but Bent, played onside by Gabriel Heintze, hesitated and Carroll plunged at his fee to save.


Use Llama Index's SimpleDirectoryReader to load all the documents. After this we will specify an embedding model from HuggingFace and add all of this to Llama Index's settings

Model 1

In [ ]:
# Use the combined text file from the previous steps
combined_file_path = "/content/combined_bbcsport.txt"
reader = FlatReader()
docs = reader.load_data(Path(combined_file_path))
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 1 docs


Model 2

In [8]:
# Use the combined text file from the previous steps
combined_file_path = "/content/combined_bbcsport.txt"
reader = FlatReader()
docs = reader.load_data(Path(combined_file_path))
print(f"Loaded {len(docs)} docs")

# Initialize a HuggingFace Embedding model
embed_model = HuggingFaceEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Use the global variable (OLLAMA_MODEL) as our LLM
OLLAMA_MODEL = os.getenv('OLLAMA_MODEL', 'phi:latest')
llm = Ollama(model=OLLAMA_MODEL, request_timeout=900.0)

# Specify the LLM and embedding model into LlamaIndex's settings
Settings.llm = llm
Settings.embed_model = embed_model

Loaded 1 docs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/70.8k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Setup a vector database (this time using Chroma DB) and index our documents (via embeddings). This will allow us to search for relevant documents to our query

In [9]:
# Create client ("db") and a database ("chroma_db")
db = chromadb.PersistentClient(path="./chroma_db")

# Create a collection/table ("demo-for-bbcsport") in the db
chroma_collection = db.create_collection("demo-for-bbcsport")

# Set up ChromaVectorStore and load in data
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# Specify Chroma as our vector db
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create the index
index = VectorStoreIndex.from_documents(
    docs,  # the abstract documents (nodes from bbcsport_nodes)
    storage_context=storage_context,
    embed_model=embed_model
)

# Print the metadata
print(chroma_collection)

# Print the name of the collection (table)
print(f'Collection name is: {chroma_collection.name}')

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


name='demo-for-bbcsport' id=UUID('485f4fa7-6336-4222-b898-9fd467c3bd66') metadata=None tenant='default_tenant' database='default_database'
Collection name is: demo-for-bbcsport


Prompt Template

In [10]:
# Set up a prompt template to ensure the LLM always answers
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

qa_prompt_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, "
    "answer the question: {query_str}\n"
)

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "Always answer the question, even if the context isn't helpful."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str),
    ]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

print(
    index.as_query_engine(
        text_qa_template=text_qa_template,
        llm=llm,
    ).query("Who is Thierry Henry?")
)

 The passage does not provide any additional context or details about who Thierry Henry is. Therefore, it is not possible to accurately determine who Thierry Henry is based on the given information alone.



Start retrieval augmented generating

In [11]:
# Assuming the necessary steps to create an index have been done
# Initialize query engine and run sample queries
query_engine = index.as_query_engine()  # use the vector db for queries

# Sample queries for testing
queries = [
    "Who won the crown of the Premier League in 2005?",
    "Describe the key players in Manchester United during the 2004-2005 season.",
    "Who is the leading goal scorers in the Premier League during the 2004-2005 season?",
    "How did Chelsea perform in the 2004-2005 football season?",
    "Provide an overview of the UEFA Champions League matches involving UK teams in 2004-2005.",
    "Detail the performance of Liverpool in the 2004-2005 season."
]

# Execute and print responses for each query
for query in queries:
    response = query_engine.query(query)  # query the model with context
    print(f"Query: {query}")
    print(f"Response: {response.response}\n")

Query: Who won the crown of the Premier League in 2005?
Response:  Based on the context information provided, we do not have enough information to accurately determine who won the Crown of the Premier League in 2005. We would need more information such as a specific team or player to answer this question definitively.


Query: Describe the key players in Manchester United during the 2004-2005 season.
Response:  Based on the given context, it is mentioned that Manchester United had an exceptional season in 2004/2005. The team's top scorer was Peter Schmeichel with a total of 94 goals, breaking his own record set in 1997. However, he missed the game against Czech Republic due to an ankle injury.


Query: Who is the leading goal scorers in the Premier League during the 2004-2005 season?
Response:  The leading goal scorers in the Premier League during the 2004-2005 season were Paul Ince (Arsenal), Wayne Rooney (Manchester United) and Steven Pemberton (West Ham United).
--------------------

Inspect the output metadata

In [12]:
response.metadata # print the full output from the LLM

{'e6e06ab1-c471-4280-a0e7-a5031d70636a': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'},
 '5622d170-e919-46ec-b446-d4430ab3d403': {'filename': 'combined_bbcsport.txt',
  'extension': '.txt'}}